## Interval Analysis

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc = nn.Linear(28*28, 200)
#         self.fc2 = nn.Linear(200,10)

#     def forward(self, x):
#         x = x.view((-1, 28*28))
#         x = F.relu(self.fc(x))
#         x = self.fc2(x)
#         x = F.softmax(x, dim=-1) # added softmax for probabilities
#         return x

# class Normalize(nn.Module):
#     def forward(self, x):
#         return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
# model = nn.Sequential(Normalize(), Net())
# model = model.to(device)


## make model compatible with the package
class NetBP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 200)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(200, 10)  # logits
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

model = NetBP().to(device)

model.train()


NetBP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)

In [2]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [3]:
train_model(model, 15)
test_model(model)

Epoch 1/15, Loss: 1.250
Epoch 2/15, Loss: 0.498
Epoch 3/15, Loss: 0.395
Epoch 4/15, Loss: 0.353
Epoch 5/15, Loss: 0.327
Epoch 6/15, Loss: 0.308
Epoch 7/15, Loss: 0.293
Epoch 8/15, Loss: 0.280
Epoch 9/15, Loss: 0.268
Epoch 10/15, Loss: 0.257
Epoch 11/15, Loss: 0.247
Epoch 12/15, Loss: 0.238
Epoch 13/15, Loss: 0.229
Epoch 14/15, Loss: 0.221
Epoch 15/15, Loss: 0.213
Accuracy on images: 94.25


### Write the interval analysis for the simple model

In [4]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation

# I used auto_LiRPA
! git clone https://github.com/Verified-Intelligence/auto_LiRPA
! pip install auto_LiRPA/.

fatal: destination path 'auto_LiRPA' already exists and is not an empty directory.
Processing ./auto_LiRPA
  Preparing metadata (setup.py) ... done
  Created wheel for auto_LiRPA: filename=auto_LiRPA-0.6.0-py3-none-any.whl size=250500 sha256=55ea274ab2a55311b46c09ba6f9d6f23a9597967c54b32f292c187f037b44051
  Stored in directory: /tmp/pip-ephem-wheel-cache-vxbqipzv/wheels/40/50/3c/d3268d5650dce1e4b6195a2b0433a92211bae5b1b40de4c53e
Successfully built auto_LiRPA
  Attempting uninstall: auto_LiRPA
    Found existing installation: auto_LiRPA 0.6.0
    Uninstalling auto_LiRPA-0.6.0:
      Successfully uninstalled auto_LiRPA-0.6.0


In [5]:
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm  # pip install auto_LiRPA

dummy = torch.zeros(1,1,28,28, device=device)
model_bp = BoundedModule(model, dummy)  # default options fine for IBP

@torch.no_grad()
def verified_accuracy_ibp(model_bp, loader, eps):
    model_bp.eval()
    total = verified = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        ptb = PerturbationLpNorm(norm=np.inf, eps=float(eps))
        x_ptb = BoundedTensor(x, ptb)

        # Direct output bounds (no C): returns [B,10] lower/upper on logits
        lb, ub = model_bp.compute_bounds(x=(x_ptb,), method='IBP')

        B = y.size(0)
        true_lb = lb[torch.arange(B, device=device), y]          # lower bound of true class
        ub_others = ub.clone()
        ub_others[torch.arange(B, device=device), y] = -1e9      # exclude true class
        max_other = ub_others.max(dim=1).values                   # max upper of other classes

        is_cert = (true_lb > max_other)                           # certified robust?
        verified += is_cert.sum().item()
        total    += B

    acc = 100.0 * verified / max(1, total)
    print(f"Verified accuracy (IBP, eps={eps}): {acc:.2f}%")
    return acc

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
import numpy as np

# Test the model on 10 L∞ neighborhoods (ε values) evenly spaced between 0.01 and 0.1 inclusive
eps_values = np.linspace(0.01, 0.1, 10)

for eps in eps_values:
    verified_accuracy_ibp(model_bp, test_loader, eps)

/usr/local/lib/python3.12/dist-packages/torchvision/datasets/mnist.py:143: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(img.numpy(), mode="L")


Verified accuracy (IBP, eps=0.01): 26.83%
Verified accuracy (IBP, eps=0.020000000000000004): 0.52%
Verified accuracy (IBP, eps=0.030000000000000006): 0.00%
Verified accuracy (IBP, eps=0.04000000000000001): 0.00%
Verified accuracy (IBP, eps=0.05000000000000001): 0.00%
Verified accuracy (IBP, eps=0.06000000000000001): 0.00%
Verified accuracy (IBP, eps=0.07): 0.00%
Verified accuracy (IBP, eps=0.08): 0.00%
Verified accuracy (IBP, eps=0.09000000000000001): 0.00%
Verified accuracy (IBP, eps=0.1): 0.00%
